In [ ]:
import base64
import requests
import os
import autogen
import pprint


In [ ]:
print(dir(autogen))
print(autogen.__file__)


In [ ]:
config_list_gpt4 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4", "gpt-4-0314", "gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
    },
)
print(config_list_gpt4)


In [ ]:
# OpenAI API Key
api_key = os.getenv("OPENAI_API_KEY")


In [37]:
# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image
def encode_image_to_base64(image_path):
    # This function reads the image file and converts it to a base64 string
    with open(image_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode()
    return encoded_string

def send_image_to_api(image_path):
    # Encode the image to base64
    base64_image = encode_image_to_base64(image_path)
  

    # Prepare headers
    headers = {
      "Content-Type": "application/json",
      "Authorization": f"Bearer {api_key}"
    }

    # Prepare payload
    payload = {
      "model": "gpt-4-vision-preview",
      "messages": [
        {
          "role": "user",
          "content": [
            {
              "type": "image_url",
              "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}"
              }
            }
          ]
        }
      ],
      "max_tokens": 300
    }

    # Send the request
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    return pprint.pprint(response.json())


In [38]:


llm_config = {
    "config_list": config_list_gpt4,
    "seed": 42,
    "request_timeout": 600,
    "temperature": 0,
    "functions": [
        {
            "name": "send_image_to_api",
            "description": "Gets image path and sends it to the API.",
            "parameters": {
                "type": "object",
                "properties": {
                    "image_path": {
                        "type": "string",
                        "description": "Valid path.",
                    }
                },
                "required": ["image_path"],
            },
        }
    ]
}
coder = autogen.AssistantAgent(
    name="Assistant",  # the default assistant agent is capable of solving problems with code
    system_message="""For file uploading functionality use functions that is registered(send_image_to_api) 
    in the user_proxy agent.""",
    llm_config=llm_config,
)
user_proxy = autogen.UserProxyAgent(
   name="user_proxy",
   is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
   human_input_mode="NEVER",
   system_message="""Reply TERMINATE if the task has been solved at full satisfaction.
   Otherwise, reply CONTINUE, or the why the task is not solved yet""",
   code_execution_config={"last_n_messages": 3, "work_dir": "groupchat"},
   llm_config=llm_config,
)

user_proxy.register_function(function_map={
    "send_image_to_api": send_image_to_api
})


In [39]:
user_proxy.initiate_chat(
    coder,
    message="""run the registered function send_image_to_api to upload file located in /Users/serkanhaytac/experimental-projects/autogen-habittracker/app_screenshot.jpg and describe the image""",
)


user_proxy (to Assistant):

run the registered function send_image_to_api to upload file located in /Users/serkanhaytac/experimental-projects/autogen-habittracker/app_screenshot.jpg and describe the image

--------------------------------------------------------------------------------
[autogen.oai.completion: 11-18 12:31:49] {788} WARNING - Completion was provided with a config_list, but the list was empty. Adopting default OpenAI behavior, which reads from the 'model' parameter instead.
Assistant (to user_proxy):

***** Suggested function Call: send_image_to_api *****
Arguments: 
{
  "image_path": "/Users/serkanhaytac/experimental-projects/autogen-habittracker/app_screenshot.jpg"
}
******************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION send_image_to_api...
{'choices': [{'finish_details': {'type': 'max_tokens'},
              'index': 0,
              'message': {'content': '